# 1. 필요 라이브러리 import

In [1]:
import pandas as pd # pandas package
import numpy as np  # numpy package 
from sklearn.preprocessing import LabelEncoder # 레이블 인코더 
from sklearn.preprocessing import OneHotEncoder # 더미 변수 생성,가변환 함수 임포트 

# 2. 데이터셋 준비

In [2]:
# 데이터 셋 생성 
data = {'이름': ['홍길동','성춘향','이몽룡','김철수','김영희','김만수','오지랖',
                 '만순이','문정억','백만원','천만원','수십억','현재미' ], 
        '몸무게': [95,85,75,70,65,55,120,100,71,65,75,100,77],
        '키': [170,170,170,170,170,175,171,165,164,177,163,192,182],
        '거주지': ['서울','대구','대전','서울','경북','서울','경북','서울',
                   '서울','전남','전북','경북','서울'],
        '주민번호': ['81XXXX-1XXXXXX','81XXXX-1XXXXXX','79XXXX-2XXXXXX','71XXXX-1XXXXXX',
                     '65XXXX-2XXXXXX','81XXXX-1XXXXXX','63XXXX-1XXXXXX','81XXXX-3XXXXXX',
                     '81XXXX-1234562','55XXXX-1234562','71XXXX-2234562','92XXXX-1234562',
                     '85XXXX-2234562'],
        '흡연여부': ['흡연','미흡연','미흡연','흡연','흡연','미흡연','흡연','흡연','흡연',
                     '미흡연','미흡연','미흡연','미흡연']
       }

df = pd.DataFrame(data)

In [3]:
# 데이터셋 확인
df.head()

,이름,몸무게,키,거주지,주민번호,흡연여부
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연


In [4]:
# 수정할 데이터프레임 복사
df_mod = df.copy()

# 3. 파생변수 생성
## 1) 이름, 흡연여부(레이블 인코딩 변수 생성)

In [5]:
# 이름 변수를 '이름_인코딩' 변수로 변환
# 레이블 인코더 생성
encoing = LabelEncoder()
# X_train 데이터를 이용하여 피팅하고 레이블 숫자로 변환
encoing.fit(df_mod['이름'])

df_mod["이름_인코딩"] = encoing.transform(df_mod["이름"])

In [6]:
print(list(encoing.classes_)) # 클래스 확인
print(list(encoing.inverse_transform([1,0]))) # 인코딩값으로 문자값 확인 

['김만수', '김영희', '김철수', '만순이', '문정억', '백만원', '성춘향', '수십억', '오지랖', '이몽룡', '천만원', '현재미', '홍길동']
['김영희', '김만수']


In [7]:
# 레이블 인코더 생성
encoing2 = LabelEncoder()
# X_train 데이터를 이용하여 피팅하고 레이블 숫자로 변환
encoing2.fit(df_mod["흡연여부"])

df_mod["흡연여부_인코딩"] = encoing2.transform(df_mod["흡연여부"])

In [8]:
df_mod.head()

,이름,몸무게,키,거주지,주민번호,흡연여부,이름_인코딩,흡연여부_인코딩
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연,12,1
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연,6,0
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연,9,0
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연,2,1
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연,1,1


## 2) 나이 변수 생성

In [9]:
# 주민번호로 나이 계산 함수 정의
CurYear = 2022

def stdInfo(rrn) :
    # 앞 2자리 이용하여  나이 계산
    if int(rrn[:2]) < 21 and int(rrn[7]) in (3,4) :
        biryear = 2000 + int(rrn[:2])
    else :
        biryear = 1900 + int(rrn[:2])
    return CurYear - biryear

In [10]:
df_mod["나이"] = df_mod['주민번호'].apply(stdInfo)

In [12]:
df_mod.head()

,이름,몸무게,키,거주지,주민번호,흡연여부,이름_인코딩,흡연여부_인코딩,나이
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연,12,1,41
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연,6,0,41
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연,9,0,43
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연,2,1,51
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연,1,1,57


## 3) 성별 변수 생성

In [13]:
# 성별 계산 함수정의
def sex(rrn) :
    # 7자리 이용하여 성별 계산
    if int(rrn[7]) == 1 or int(rrn[7]) == 3 :
        gen = '남'
    else :
        gen = '여'
    return gen

In [14]:
df_mod['성별'] = df_mod['주민번호'].apply(sex)

In [15]:
# 성별 One-Hot encoding

# sparsse를 True로 할 경우 "(행, 열) 1"의 좌표리스트의 형식, False로 할 경우 넘파이 배열로 반환
one_encoding = OneHotEncoder(sparse=False)

# 가변환된 값을 x_train_one 데이터프레임으로 저장
df_mod_one = pd.DataFrame(one_encoding.fit_transform(df_mod[['성별']]), 
                         columns=['성별_남', '성별_여'])

In [16]:
df_mod = pd.concat([df_mod, df_mod_one], axis=1)

In [17]:
df_mod.head()

,이름,몸무게,키,거주지,주민번호,흡연여부,이름_인코딩,흡연여부_인코딩,나이,성별,성별_남,성별_여
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연,12,1,41,남,1.0,0.0
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연,6,0,41,남,1.0,0.0
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연,9,0,43,여,0.0,1.0
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연,2,1,51,남,1.0,0.0
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연,1,1,57,여,0.0,1.0


## 4) 나이 변수의 범주화 변수 생성(연속형 변수의 범주형 변수 변환)
### - pd.cut()함수 이용 : 연속된 수치(continuous values)를 구간으로 나누어 카테고리화 할 때 사용

In [18]:
df_mod['나이_범주'] = pd.cut(df_mod.나이, bins=[0, 10, 20, 30, 40, 50, 60, 70],
                        labels=[0, 10, 20, 30, 40, 50, 60])

In [19]:
df_mod.head()

,이름,몸무게,키,거주지,주민번호,흡연여부,이름_인코딩,흡연여부_인코딩,나이,성별,성별_남,성별_여,나이_범주
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연,12,1,41,남,1.0,0.0,40
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연,6,0,41,남,1.0,0.0,40
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연,9,0,43,여,0.0,1.0,40
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연,2,1,51,남,1.0,0.0,50
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연,1,1,57,여,0.0,1.0,50


## 5) BMI 변수 생성
### - BMI : 체질량 지수, 인간의 비만도를 나타내는 지수로, 체중과 키의 관계로  계산
### - eval()함수 이용 : 문자열로 된 수식을 input으로 받아 그 결과를 return하는 함수

In [20]:
# BMI 계산
df_mod.eval('BMI = 몸무게 / ((키/100) * (키/100))', inplace=True)

In [21]:
df_mod.head()

,이름,몸무게,키,거주지,주민번호,흡연여부,이름_인코딩,흡연여부_인코딩,나이,성별,성별_남,성별_여,나이_범주,BMI
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연,12,1,41,남,1.0,0.0,40,32.871972
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연,6,0,41,남,1.0,0.0,40,29.411765
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연,9,0,43,여,0.0,1.0,40,25.951557
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연,2,1,51,남,1.0,0.0,50,24.221453
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연,1,1,57,여,0.0,1.0,50,22.491349


In [22]:
# BMI 분류
df_mod['BMI분류'] = pd.cut(df_mod.BMI, bins=[0, 18, 23, 25, 30, 90],
                        labels=['저체중', '정상', '과체중', '비만', '고도비만'])

In [23]:
df_mod['BMI분류코드'] = df_mod['BMI분류'].map({ "저체중" : 0, "정상" : 1, "과체중" : 2, "비만" : 3, "고도비만" : 4})

## 6) 최종 필요한 변수만 데이터프레임 생성

In [24]:
# 현재까지 추가된 변수 확인
df_mod.head()

,이름,몸무게,키,거주지,주민번호,흡연여부,이름_인코딩,흡연여부_인코딩,나이,성별,성별_남,성별_여,나이_범주,BMI,BMI분류,BMI분류코드
0,홍길동,95,170,서울,81XXXX-1XXXXXX,흡연,12,1,41,남,1.0,0.0,40,32.871972,고도비만,4
1,성춘향,85,170,대구,81XXXX-1XXXXXX,미흡연,6,0,41,남,1.0,0.0,40,29.411765,비만,3
2,이몽룡,75,170,대전,79XXXX-2XXXXXX,미흡연,9,0,43,여,0.0,1.0,40,25.951557,비만,3
3,김철수,70,170,서울,71XXXX-1XXXXXX,흡연,2,1,51,남,1.0,0.0,50,24.221453,과체중,2
4,김영희,65,170,경북,65XXXX-2XXXXXX,흡연,1,1,57,여,0.0,1.0,50,22.491349,정상,1


In [25]:
data_pre = df_mod[['나이_범주', '성별_남', '성별_여', '흡연여부_인코딩', '키', '몸무게', 'BMI분류코드']]

In [26]:
data_pre.head()

,나이_범주,성별_남,성별_여,흡연여부_인코딩,키,몸무게,BMI분류코드
0,40,1.0,0.0,1,170,95,4
1,40,1.0,0.0,0,170,85,3
2,40,0.0,1.0,0,170,75,3
3,50,1.0,0.0,1,170,70,2
4,50,0.0,1.0,1,170,65,1
